# ARCA Beverage Demo: Feature Store Implementation

## Overview
This notebook demonstrates Snowflake's Feature Store capabilities with **multiple refresh frequencies**:
- **Hourly**: Temporal features (week_of_year, month, is_holiday_week)
- **Daily**: Customer behavior features (purchase patterns, recency, quartiles)
- **Weekly**: Product features (trends, seasonal demand, category performance)

## Key Message
**Feature Store is completely independent of your models.**  
You can have daily, weekly, monthly models - all consuming from the same centralized Feature Store.  
Feature refresh frequency is configurable and doesn't limit your model granularity.

In [1]:
from snowflake.snowpark.context import get_active_session
from snowflake.ml.feature_store import FeatureStore, Entity, FeatureView
from snowflake.snowpark import functions as F, Window
from datetime import datetime, timedelta
import json

# Use active Snowsight session (no credentials needed)
session = get_active_session()

# Set context
session.sql("USE WAREHOUSE ARCA_DEMO_WH").collect()
session.sql("USE DATABASE ARCA_BEVERAGE_DEMO").collect()
session.sql("USE SCHEMA ML_DATA").collect()

print(f"✅ Connected to Snowflake")
print(f"   Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database: {session.get_current_database()}")
print(f"   Schema: {session.get_current_schema()}")

## 1. Create Feature Store Schema

In [4]:
from snowflake.ml.feature_store import CreationMode

session.sql("CREATE SCHEMA IF NOT EXISTS ARCA_BEVERAGE_DEMO.FEATURE_STORE").collect()
session.sql("USE SCHEMA ARCA_BEVERAGE_DEMO.FEATURE_STORE").collect()

fs = FeatureStore(
    session=session,
    database="ARCA_BEVERAGE_DEMO",
    name="FEATURE_STORE",
    default_warehouse="ARCA_DEMO_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

print("✅ Feature Store initialized")

## 2. Define Entities

Entities represent the business objects we're building features for:

In [4]:
customer_entity = Entity(
    name="CUSTOMER",
    join_keys=["CUSTOMER_ID"],
    desc="Customer entity for beverage sales forecasting"
)

fs.register_entity(customer_entity)
print("✅ Customer entity registered")

product_entity = Entity(
    name="PRODUCT",
    join_keys=["PRODUCT_ID"],
    desc="Product entity for beverage catalog"
)

fs.register_entity(product_entity)
print("✅ Product entity registered")

## 3. Feature View A: Temporal Features (Hourly Refresh)

**Refresh Frequency: 1 HOUR**  
These are date-based features that change frequently and are used by all models.

In [5]:
temporal_df = session.sql("""
    SELECT DISTINCT
        TRANSACTION_DATE AS FEATURE_DATE,
        TRANSACTION_DATE AS TIMESTAMP_COL,
        WEEKOFYEAR(TRANSACTION_DATE) AS WEEK_OF_YEAR,
        MONTH(TRANSACTION_DATE) AS MONTH,
        QUARTER(TRANSACTION_DATE) AS QUARTER,
        DAYOFWEEK(TRANSACTION_DATE) AS DAY_OF_WEEK,
        CASE 
            WHEN MONTH(TRANSACTION_DATE) IN (12, 1) THEN 1
            WHEN MONTH(TRANSACTION_DATE) IN (3, 4) THEN 1
            WHEN MONTH(TRANSACTION_DATE) = 9 THEN 1
            ELSE 0
        END AS IS_HOLIDAY_SEASON,
        CASE 
            WHEN DAYOFWEEK(TRANSACTION_DATE) IN (0, 6) THEN 1 
            ELSE 0 
        END AS IS_WEEKEND
    FROM ARCA_BEVERAGE_DEMO.ML_DATA.TRANSACTIONS
    WHERE TRANSACTION_DATE >= DATEADD(YEAR, -2, CURRENT_DATE())
""")

temporal_fv = FeatureView(
    name="TEMPORAL_FEATURES",
    entities=[],
    feature_df=temporal_df,
    timestamp_col="TIMESTAMP_COL",
    refresh_freq="1 hour",
    desc="Hourly-refreshed temporal features: week_of_year, month, is_holiday_season, day_of_week"
)

temporal_fv = fs.register_feature_view(
    feature_view=temporal_fv,
    version="v2",
    block=True
)

print("✅ Temporal Features registered (Hourly refresh)")

## 4. Feature View B: Customer Behavior Features (Daily Refresh)

**Refresh Frequency: 1 DAY**  
Customer purchase patterns that update daily:
- Average weekly purchase amount (rolling 4 weeks)
- Purchase frequency last 4 weeks
- Days since last purchase
- Customer volume quartile

In [9]:
customer_behavior_df = session.sql("""
    WITH date_spine AS (
        SELECT DISTINCT DATE_TRUNC('WEEK', TRANSACTION_DATE) AS FEATURE_DATE
        FROM ARCA_BEVERAGE_DEMO.ML_DATA.TRANSACTIONS
        WHERE TRANSACTION_DATE >= '2024-01-01'
    ),
    customer_agg AS (
        SELECT
            ds.FEATURE_DATE,
            t.CUSTOMER_ID,
            AVG(t.REVENUE) AS AVG_WEEKLY_PURCHASE_AMOUNT,
            COUNT(DISTINCT DATE_TRUNC('WEEK', t.TRANSACTION_DATE)) AS PURCHASE_FREQUENCY_LAST_4_WEEKS,
            DATEDIFF(DAY, MAX(t.TRANSACTION_DATE), ds.FEATURE_DATE) AS DAYS_SINCE_LAST_PURCHASE,
            SUM(t.UNITS_SOLD) AS TOTAL_UNITS_4_WEEKS,
            COUNT(DISTINCT t.TRANSACTION_DATE) AS PURCHASE_DAYS_LAST_4_WEEKS,
            AVG(t.UNITS_SOLD) AS AVG_UNITS_PER_TRANSACTION
        FROM date_spine ds
        JOIN ARCA_BEVERAGE_DEMO.ML_DATA.TRANSACTIONS t
            ON t.TRANSACTION_DATE >= DATEADD(DAY, -28, ds.FEATURE_DATE)
            AND t.TRANSACTION_DATE < ds.FEATURE_DATE
        GROUP BY ds.FEATURE_DATE, t.CUSTOMER_ID
    ),
    volume_quartiles AS (
        SELECT
            FEATURE_DATE,
            CUSTOMER_ID,
            TOTAL_UNITS_4_WEEKS,
            NTILE(4) OVER (PARTITION BY FEATURE_DATE ORDER BY TOTAL_UNITS_4_WEEKS) AS CUSTOMER_VOLUME_QUARTILE
        FROM customer_agg
    )
    SELECT
        ca.CUSTOMER_ID,
        ca.FEATURE_DATE AS FEATURE_TIMESTAMP,
        ca.AVG_WEEKLY_PURCHASE_AMOUNT,
        ca.PURCHASE_FREQUENCY_LAST_4_WEEKS,
        ca.DAYS_SINCE_LAST_PURCHASE,
        vq.CUSTOMER_VOLUME_QUARTILE,
        ca.PURCHASE_DAYS_LAST_4_WEEKS,
        ca.AVG_UNITS_PER_TRANSACTION,
        CASE 
            WHEN ca.PURCHASE_FREQUENCY_LAST_4_WEEKS >= 4 THEN 'VERY_ACTIVE'
            WHEN ca.PURCHASE_FREQUENCY_LAST_4_WEEKS >= 2 THEN 'ACTIVE'
            WHEN ca.PURCHASE_FREQUENCY_LAST_4_WEEKS >= 1 THEN 'MODERATE'
            ELSE 'INACTIVE'
        END AS ACTIVITY_LEVEL
    FROM customer_agg ca
    JOIN volume_quartiles vq 
        ON ca.CUSTOMER_ID = vq.CUSTOMER_ID 
        AND ca.FEATURE_DATE = vq.FEATURE_DATE
""")

customer_behavior_fv = FeatureView(
    name="CUSTOMER_BEHAVIOR_FEATURES",
    entities=[customer_entity],
    feature_df=customer_behavior_df,
    timestamp_col="FEATURE_TIMESTAMP",
    refresh_freq="1 day",
    desc="Daily-refreshed customer behavior: purchase patterns, recency, volume quartiles"
)

customer_behavior_fv = fs.register_feature_view(
    feature_view=customer_behavior_fv,
    version="v2",
    block=True
)

print("✅ Customer Behavior Features registered (Daily refresh)")

## 5. Feature View C: Product Features (Weekly Refresh)

**Refresh Frequency: 7 DAYS**  
Product-level trends and performance:
- Product trend last 4 weeks
- Seasonal demand factor
- Category performance score

In [7]:
product_features_df = session.sql("""
    WITH product_sales_4w AS (
        SELECT
            PRODUCT_ID,
            TRANSACTION_DATE,
            SUM(UNITS_SOLD) AS DAILY_UNITS,
            SUM(REVENUE) AS DAILY_REVENUE
        FROM ARCA_BEVERAGE_DEMO.ML_DATA.TRANSACTIONS
        WHERE TRANSACTION_DATE >= DATEADD(WEEK, -4, CURRENT_DATE())
        GROUP BY PRODUCT_ID, TRANSACTION_DATE
    ),
    product_sales_8w AS (
        SELECT
            PRODUCT_ID,
            SUM(UNITS_SOLD) AS UNITS_5_8_WEEKS
        FROM ARCA_BEVERAGE_DEMO.ML_DATA.TRANSACTIONS
        WHERE TRANSACTION_DATE BETWEEN DATEADD(WEEK, -8, CURRENT_DATE()) 
              AND DATEADD(WEEK, -4, CURRENT_DATE())
        GROUP BY PRODUCT_ID
    ),
    category_performance AS (
        SELECT
            p.CATEGORY,
            SUM(t.UNITS_SOLD) AS CATEGORY_UNITS,
            AVG(t.REVENUE) AS CATEGORY_AVG_REVENUE
        FROM ARCA_BEVERAGE_DEMO.ML_DATA.PRODUCTS p
        JOIN ARCA_BEVERAGE_DEMO.ML_DATA.TRANSACTIONS t ON p.PRODUCT_ID = t.PRODUCT_ID
        WHERE t.TRANSACTION_DATE >= DATEADD(WEEK, -4, CURRENT_DATE())
        GROUP BY p.CATEGORY
    )
    SELECT
        p.PRODUCT_ID,
        CURRENT_DATE() AS FEATURE_TIMESTAMP,
        COALESCE(SUM(ps4.DAILY_UNITS), 0) AS UNITS_LAST_4_WEEKS,
        COALESCE(AVG(ps4.DAILY_UNITS), 0) AS AVG_DAILY_UNITS,
        CASE 
            WHEN COALESCE(ps8.UNITS_5_8_WEEKS, 0) = 0 THEN 0
            ELSE (COALESCE(SUM(ps4.DAILY_UNITS), 0) - COALESCE(ps8.UNITS_5_8_WEEKS, 0)) / 
                 NULLIF(COALESCE(ps8.UNITS_5_8_WEEKS, 0), 0)
        END AS PRODUCT_TREND_LAST_4_WEEKS,
        CASE 
            WHEN MONTH(CURRENT_DATE()) IN (12, 1, 6, 7) THEN 1.3
            WHEN MONTH(CURRENT_DATE()) IN (3, 4, 9, 10) THEN 1.1
            ELSE 1.0
        END AS SEASONAL_DEMAND_FACTOR,
        cp.CATEGORY_UNITS / NULLIF(SUM(cp.CATEGORY_UNITS) OVER (), 0) AS CATEGORY_PERFORMANCE_SCORE,
        p.CATEGORY,
        p.PRICE
    FROM ARCA_BEVERAGE_DEMO.ML_DATA.PRODUCTS p
    LEFT JOIN product_sales_4w ps4 ON p.PRODUCT_ID = ps4.PRODUCT_ID
    LEFT JOIN product_sales_8w ps8 ON p.PRODUCT_ID = ps8.PRODUCT_ID
    LEFT JOIN category_performance cp ON p.CATEGORY = cp.CATEGORY
    GROUP BY p.PRODUCT_ID, p.CATEGORY, p.PRICE, ps8.UNITS_5_8_WEEKS, 
             cp.CATEGORY_UNITS, cp.CATEGORY_AVG_REVENUE
""")

product_features_fv = FeatureView(
    name="PRODUCT_FEATURES",
    entities=[product_entity],
    feature_df=product_features_df,
    timestamp_col="FEATURE_TIMESTAMP",
    refresh_freq="7 days",
    desc="Weekly-refreshed product features: trends, seasonal demand, category performance"
)

product_features_fv = fs.register_feature_view(
    feature_view=product_features_fv,
    version="v2",
    block=True
)

print("✅ Product Features registered (Weekly refresh)")

## 6. Verify Feature Store Setup

In [5]:
all_fvs = fs.list_feature_views()
print("\n📊 Registered Feature Views:")
print(all_fvs.select("NAME", "VERSION", "DESC", "REFRESH_FREQ", "ENTITIES").show())

all_entities = fs.list_entities()
print("\n📊 Registered Entities:")
print(all_entities.show())

## 7. Feature Retrieval Examples

Demonstrate how to retrieve features for different model granularities:

In [7]:
# Get the registered FeatureView object
customer_fv = fs.get_feature_view(
    name="CUSTOMER_BEHAVIOR_FEATURES",
    version="v2"
)

spine_df = session.sql("""
    SELECT DISTINCT
        CUSTOMER_ID,
        DATE_TRUNC('WEEK', TRANSACTION_DATE) AS WEEK_START_DATE
    FROM ARCA_BEVERAGE_DEMO.ML_DATA.TRANSACTIONS
    WHERE IS_INFERENCE = FALSE
    LIMIT 100
""")

enriched_df = fs.retrieve_feature_values(
    spine_df=spine_df,
    features=[customer_fv],
    spine_timestamp_col="WEEK_START_DATE"
)

print("\n✅ Sample enriched training data with features:")
enriched_df.show(5)

## 8. Feature Store Summary

### ✅ Completed Setup:
1. **3 Feature Views** with different refresh frequencies (1 hour, 1 day, 7 days)
2. **2 Entities** (Customer, Product)
3. **Point-in-time correctness** for historical training data
4. **Incremental refresh** for efficient feature computation

### 🎯 Key Capabilities Demonstrated:
- ✅ Multiple refresh frequencies in single Feature Store
- ✅ Independent feature computation from model granularity
- ✅ Centralized feature definitions
- ✅ Automatic feature versioning

### 📈 Next Steps:
1. **Customer Segmentation** (Notebook 03)
2. **Many Model Training** (Notebook 04)
3. **Model Registry & Versioning** (Notebook 05)
4. **ML Observability** (Notebook 06)

In [8]:
print("\n🎉 Feature Store setup complete!")
print("\n📝 Summary:")
print(f"  - Temporal Features: Hourly refresh")
print(f"  - Customer Behavior Features: Daily refresh")
print(f"  - Product Features: Weekly refresh")
print(f"\n💡 These features can now be consumed by models at ANY granularity!")